<a href="https://colab.research.google.com/github/tbaeumel/transformer-digit-arithmetic/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get the data from the repo -> anonymize!

In [ ]:
!git clone https://github.com/tbaeumel/transformer-digit-arithmetic.git

In [ ]:
%cd transformer-digit-arithmetic

Get pyvene
Olmo2 was not in pyvene - i wrote an extension so that olmo2 can be an intervenable model.
 has to be anonymized

In [ ]:
# pip uninstall -y pyvene  # pyvene has to be uninstalled if a standard version is currently installed
# pip install pyvene
!pip install git+https://github.com/tbaeumel/pyvene

In [ ]:
import pyvene as pv
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import sys
import torch.nn.functional as F
from pyvene import embed_to_distrib, top_vals, format_token
from pyvene import (
    ZeroIntervention,
    IntervenableModel,
    VanillaIntervention, Intervention,
    RepresentationConfig,
    IntervenableConfig,
    ConstantSourceIntervention,
    LocalistRepresentationIntervention
)
from huggingface_hub import login
import re
import numpy
from tqdm import tqdm
from matplotlib import pyplot as plt
import transformers
import os

## Load model

Llama 70B wont work on colab

llama 8b and olmo 2 7b will work on l4 or a100

people have to login with their huggingface key to get access to the llama models

In [ ]:
# Login to Huggingface to get access to model parameters
# Paste your token here
login('')

In [ ]:
# choose your model
model_name = "Llama-3-8B" # Llama-3-70B # Olmo-2-7B

In [ ]:
models = {"Llama-3-8B": "meta-llama/Meta-Llama-3-8B", "Llama-3-70B": "meta-llama/Meta-Llama-3-70B", "Olmo-2-7B": "allenai/OLMo-2-1124-7B" }

# Set parameters
params = {
    'model_name': models[model_name],
    'device': "cuda" if torch.cuda.is_available() else "cpu"
}

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(params['model_name'])
model = AutoModelForCausalLM.from_pretrained(params['model_name'], torch_dtype=torch.float16).to(params['device'])

# Confirm device
print(f"Using device: {params['device']}")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
  print("Using GPU")